<div style="background-color:#000;"><img src="pqn.png"></img></div>

This notebook analyzes treasury rates by performing eigenvalue decomposition on their covariance matrix. It retrieves treasury rates, calculates their covariance, and uses eigenvalue decomposition to find principal components. The notebook verifies the orthogonality of eigenvectors and extracts the first four principal components. Then, it generates random shocks to simulate their effects on the rates. This is useful for understanding the impact of various factors on interest rates.

In [ ]:
from openbb import obb
import numpy as np
import pandas as pd

Retrieve treasury rates from the OpenBB API, convert to DataFrame, drop NaNs, and convert to decimal format.

In [ ]:
rates = (
    obb
    .fixedincome
    .government
    .treasury_rates(
        start_date="2020-01-01"
    )
    .to_df()
    .dropna()
    .div(100)
)

Display the rates DataFrame.

In [ ]:
rates

Calculate the covariance matrix of the treasury rates.

In [ ]:
C = rates.cov()

Perform eigenvalue decomposition on the covariance matrix to obtain eigenvalues and eigenvectors.

In [ ]:
eigenvalues, eigenvectors = np.linalg.eig(C)
lambda_sqrt = np.sqrt(eigenvalues)
eigv_decom = np.diag(lambda_sqrt)

Verify that the eigenvectors are orthogonal by checking if their dot product is the identity matrix.

In [ ]:
np.allclose(
    eigenvectors.T @ eigenvectors,
    np.eye(eigenvectors.shape[0])
)

Extract the first four principal components and create a DataFrame with meaningful labels.

In [ ]:
B = eigv_decom @ eigenvectors.T
B = pd.DataFrame(
    data=B[:4] * 100,
    index=["wiggle", "flex", "twist", "shift"],
    columns=rates.columns
)

Display the principal components DataFrame.

In [ ]:
B

Generate random shocks from a normal distribution to simulate changes in the rates.

In [ ]:
random_shocks = np.random.normal(0, 1, size=(4,))
random_shocks @ B

Define standard deviation and correlation matrices for another set of computations.

In [ ]:
S = np.array([0.051, 0.052, 0.061, 0.054])
R = np.array([[1, 0.61, 0.42, 0.31],
              [0.61, 1, 0.83, 0.67],
              [0.42, 0.83, 1, 0.88],
              [0.31, 0.67, 0.88, 1]])

Calculate the covariance matrix using the standard deviation and correlation matrices.

In [ ]:
C = np.diag(S) @ R @ np.diag(S)

Display the calculated covariance matrix.

In [ ]:
C

Perform eigenvalue decomposition on the new covariance matrix to obtain eigenvalues and eigenvectors.

In [ ]:
eigenvalues, eigenvectors = np.linalg.eig(C)
lambda_sqrt = np.sqrt(eigenvalues)
eigv_decom = np.diag(lambda_sqrt)

Display the eigenvectors obtained from the decomposition.

In [ ]:
eigenvectors

Verify that the new eigenvectors are orthogonal by checking if their dot product is the identity matrix.

In [ ]:
np.allclose(eigenvectors.T @ eigenvectors, np.eye(eigenvectors.shape[0]))

Extract the principal components for the new covariance matrix.

In [ ]:
B = eigv_decom @ eigenvectors.T

Display the new principal components.

In [ ]:
B

Simulate changes in rates using random shocks and basis point adjustments.

In [ ]:
m = 2
random_shocks = np.random.normal(0, 1, size=(4,))
random_shocks[2:] *= m  # Apply magnitude to z3 and z4
delta_r = random_shocks @ (100 * B)  # Basis point shocks for each rate point

Display the simulated rate changes.

In [ ]:
delta_r

<a href="https://pyquantnews.com/">PyQuant News</a> is where finance practitioners level up with Python for quant finance, algorithmic trading, and market data analysis. Looking to get started? Check out the fastest growing, top-selling course to <a href="https://gettingstartedwithpythonforquantfinance.com/">get started with Python for quant finance</a>. For educational purposes. Not investment advise. Use at your own risk.